In [7]:
import torch
print(torch.version.cuda)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,TextDataset,Trainer,TrainingArguments
import peft
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import Dataset
import transformers

12.1

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('D')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: 6.1.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpytorch_cuda.md
CUDA SETUP: Loading

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [4]:
with open("d:/test/chat/chat_data_tok.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

In [5]:
lines[:5]

['0\n',
 '[NAME] iPhone dila do Modi ji 📱🤳📱 [MESSAGE] \u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.\n',
 '[NAME] Vijay Pune [MESSAGE] \u200eVijay Pune changed the group name to “PubG Team”\n',
 '[NAME] None [MESSAGE] None\n',
 '[NAME] None [MESSAGE] None\n']

In [4]:


# Check the number of available GPUs
num_gpus = torch.cuda.device_count()

if num_gpus > 0:
    # Get details of the GPU at index 0
    gpu_0 = torch.cuda.get_device_properties(0)
    print("GPU 0 Details:")
    print(f"Name: {gpu_0.name}")
    print(f"Total Memory: {gpu_0.total_memory / (1024 ** 2):.2f} MB")
    print(f"Compute Capability: {gpu_0.major}.{gpu_0.minor}")
else:
    print("No GPUs available.")
import accelerate
print(f"accelretae version",{accelerate.__version__})
import bitsandbytes

print(bitsandbytes.__version__)

GPU 0 Details:
Name: NVIDIA GeForce GTX 1050 Ti
Total Memory: 4095.81 MB
Compute Capability: 6.1
accelretae version {'0.23.0'}

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('D')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: 6.1.
CUDA SETUP: To manually override the PyTorch CUDA 

RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [32]:
# Step 1: Load the pre-trained GPT-2 model
model_checkpoint='d:\huggingface_models\gpt2-large'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, quantization_config=bnb_config)
#model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [7]:
# Step 2: Tokenize the training data
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer.add_special_tokens({'additional_special_tokens': ['[NAME]']})
tokenizer.add_special_tokens({'additional_special_tokens': ['[MESSAGE]']})
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [12]:
#to view first 5 tokenised sentences
for i in lines[:5]:
    print(tokenizer(i,return_tensors='pt'))

{'input_ids': tensor([[ 15, 198]]), 'attention_mask': tensor([[1, 1]])}
{'input_ids': tensor([[50257,  7133,   288, 10102,   466, 14637,   474,    72, 12520,   241,
           109,  8582,    97,   111,  8582,   241,   109,   220, 50258, 24398,
         36479,  1095,   290,  3848,   389,   886,    12,  1462,    12,   437,
         19365,    13,  1400,   530,  2354,   286,   428,  8537,    11,   407,
           772, 37666,    11,   460,  1100,   393,  6004,   284,   606,    13,
           198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}
{'input_ids': tensor([[50257, 49219,   323,   350,  1726,   220, 50258, 24398,    53,  2926,
           323,   350,  1726,  3421,   262,  1448,  1438,   284,   564,   250,
         14876,    38,  4816,   447,   251,   198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [14]:
# Step 3: Prepare the training data
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="d:/test/chat/chat_in_txt.txt",
    block_size=10
)

d:\ProgramData\anaconda3\envs\torch_cuda12.1\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [26]:
dataset[:5]

tensor([[   50,  2194,   197, 12837,   198,   198,     1,   198,    53,  2926],
        [  323,   350,  1726,   197,     1, 48261,    53,  2926,   323,   350],
        [ 1726,  3421,   262,  1448,  1438,   284,   564,   250, 14876,    38],
        [ 4816,   447,   251,   198,     1,   198,    53,  2926,   323,   350],
        [ 1726,   197,     1, 14618,   284,   674,   649,   664,    84,   578]])

In [27]:
train_size = int(0.9 * len(dataset))
eval_size = len(dataset) - train_size

In [28]:
from torch.utils.data import random_split
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])
print("Training dataset size:", len(train_dataset))
print("Evaluation dataset size:", len(eval_dataset))

Training dataset size: 230290
Evaluation dataset size: 25588


In [8]:
import peft


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('D')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: 6.1.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpytorch_cuda.md
CUDA SETUP: Loading bina

d:\ProgramData\anaconda3\envs\torch_cuda12.1\Lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
d:\ProgramData\anaconda3\envs\torch_cuda12.1\Lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: D:\ProgramData\anaconda3\envs\torch_cuda12.1 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
d:\ProgramData\anaconda3\envs\torch_cuda12.1\Lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!                     If you run into issues with 8-bit matmul, you can try 4-bit quantization: https://huggingface.co/blog/4bit-transformers-bitsandbytes
  warn(msg)


RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues